In [2]:
import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
from collections import defaultdict
from utils.utils import prepare, convert_tokens_to_words
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Slicing_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.ESM import ESM
from models.TTMS import TTMS
 
from models.Modules.Attention import MultiheadAttention

loss = nn.NLLLoss()

m = AutoModel.from_pretrained('microsoft/deberta-base', cache_dir='../../../Data/bert_cache')
# t = AutoTokenizer.from_pretrained('microsoft/deberta-base', cache_dir='../../../Data/bert_cache')

In [7]:
attn_mask = torch.tensor([[1.,1,1,1,0],[1,0,0,0,0]])
hidden_states = torch.rand((2,3,4))
extended_attn_mask = m.encoder.get_attention_mask(attn_mask)
rel_pos = m.encoder.get_rel_pos(hidden_states)

rel_pos, extended_attn_mask

In [3]:
# config.reducer = 'entity'
# config.embedding = 'deberta'
# config.bert = 'microsoft/deberta-base'
# config.device = 0

manager = Manager(config)
loaders = prepare(manager)
x1 = list(loaders[0])[0]
x2 = list(loaders[1])[0]

[2021-09-08 12:07:26,355] INFO (utils.utils) Hyper Parameters are 
scale:demo
batch_size:5
k:5
threshold:-inf
signal_length:100
his_size:50
impr_size:10
lr:0.0001
bert_lr:3e-05
hidden_dim:384
world_size:0
step:0
ascend_history:False
no_dedup:False
diversify:False
granularity:avg
no_sep_his:False
no_order_embed:False
bert:bert-base-uncased
[2021-09-08 12:07:26,356] INFO (utils.utils) preparing dataset...
[2021-09-08 12:07:26,360] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_train/10/behaviors..pkl
[2021-09-08 12:07:26,373] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-09-08 12:07:27,189] INFO (utils.utils) deduplicating...
[2021-09-08 12:07:28,670] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_dev/10/behaviors..pkl
[2021-09-08 12:07:28,712] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdem

In [14]:
embedding = BERT_Embedding(manager)

encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
# encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

reducer = Matching_Reducer(manager)
# reducer = Slicing_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

model = TTMS(manager, embedding, encoderN, encoderU, reducer).to(manager.device)

In [15]:
a,b = model(x1)

tensor([[[0.3368, 0.3549, 0.3033,  ...,   -inf,   -inf,   -inf],
         [0.3194, 0.3795, 0.3507,  ...,   -inf,   -inf,   -inf],
         [0.3151, 0.3332, 0.3297,  ...,   -inf,   -inf,   -inf],
         ...,
         [0.9985, 0.9985, 0.9985,  ...,   -inf,   -inf,   -inf],
         [0.9985, 0.9985, 0.9985,  ...,   -inf,   -inf,   -inf],
         [0.9985, 0.9985, 0.9985,  ...,   -inf,   -inf,   -inf]],

        [[0.4536, 0.4767, 0.4908,  ...,   -inf,   -inf,   -inf],
         [0.4972, 0.4764, 0.4597,  ...,   -inf,   -inf,   -inf],
         [0.4207, 0.4683, 0.4715,  ...,   -inf,   -inf,   -inf],
         ...,
         [0.4426, 0.4748, 0.4656,  ...,   -inf,   -inf,   -inf],
         [0.4551, 0.4502, 0.4448,  ...,   -inf,   -inf,   -inf],
         [0.4485, 0.4862, 0.4810,  ...,   -inf,   -inf,   -inf]],

        [[0.3354, 0.3199, 0.3452,  ...,   -inf,   -inf,   -inf],
         [0.3481, 0.3353, 0.3381,  ...,   -inf,   -inf,   -inf],
         [0.3290, 0.3642, 0.3817,  ...,   -inf,   -inf,   

In [58]:
ls = loss(a,target=x1['label'])
ls.backward()

In [ ]:
d.grad

tensor([[-0.2000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2000,  0.0000,  0.0000,  0.0000,  0.0000]])